# Lesson 5 - Creating a prior Monte Carlo model simulation 

The parameter estimation process begins with a prior Monte Carlo simulation, where the user creates an ensemble of model simulations based on initial draw from model parameters specified for calibration within the bound introduced by the user. 
PEST++ samples prior parameters from a normal distribution with a mean equal to the default parameter value and a standard deviation set to a quarter of the parameter range. 
Consequently, default values should ideally fall within the parameter range and not be too close to the lower or upper bounds.

A benefit of performing a MC before running iES is the rejection sampling. Rejection sampling is a technique aimed at filtering out underperforming ensemble members. Typically, these members exhibit unfavorable parameter combinations that may detract from the trajectory of PEST++'s optimal solution in particular by confounding the information obtained by estimating the heuristic Jacobian. Hence, it proves advantageous to reject them from the ensemble before progressing further. The extent of this rejection is subjective and contingent upon modeler discretion.

In [ ]:
import pandas as pd
import pyemu
import sys
import shutil
import os
import numpy as np
from pathlib import Path

## **Step 1. Create prior Monte Carlo run directory**

Given the information needed for the Monte Carlo run is similar to the Single Model run and the only difference is in the PEST++ control file content, we copy the files in *Single_Model_Run* directory and name it *Prior_MC_Model_Run* in this lesson.

In [ ]:
%%bash
# create the run directory for lesson 5
mkdir ~/wrf-hydro-training/output/lesson5

# copy the run directory from lesson 4 to 5 to avoid compiling a run directory
cp -rf ~/wrf-hydro-training/output/lesson4/Single_Model_Run/ ~/wrf-hydro-training/output/lesson5/Prior_MC_Model_Run

# let's check out the content of the folder 
ls ~/wrf-hydro-training/output/lesson5/Prior_MC_Model_Run

## **Step 2. Make modification to the PEST++ control file** 

There are two options in the PEST++ control file that needs modification: 
* noptmax : this option is set to 0 for the single model run and needs to change to -1 to indicate we are targeting to perform a Monte Carlo model simulation. 
* ies_num_reals: number of realization to run which we want to set to 30 ensembels for this lesson. 

There are two ways to do this modification, an easy method would be to edit the `wrfpst.pst` control file directly, the other would be to use the capabilities of `PyEMU` to read an existing control file and build another based on it. Here, we attemp to show case the second approach. 

In [ ]:
# define the path to the host folder from a single model run 
wkdir = Path('/home/docker/wrf-hydro-training/output/lesson4/host') 

# define where you would like the modified PEST++ files to reside 
priordir = Path('/home/docker/wrf-hydro-training/output/lesson5/Prior_MC_Model_Run')

**NOTE:** To do this, all the files need to be relatvie to the host, if you check the content of the wrfpst.pst file, the observation file is actually an external file, so let's copy that file into the host directory before reading the control file using PyEMU. 

In [ ]:
!cp ~/wrf-hydro-training/example_case/OBS/Observation_File_20180801_20180901.csv ~/wrf-hydro-training/output/lesson4/host/Observation_File_20180801_20180901.csv 

**Read in the PESTPP control file from Lesson4**

In [ ]:
pst = pyemu.Pst(str(wkdir / 'wrfpst.pst'), resfile=str(wkdir / 'wrfpst.base.rei'))

Let's double check what parameters we have in this version of the model using pyemu (you can just look in the PEST control file too.).

In [ ]:
pst.control_data.noptmax

Now we will overwrite a few options of the control file as follows:

In [ ]:
pst.control_data.noptmax=-1
pst.pestpp_options["ies_num_reals"] = 30
pst.write(priordir/'wrfpst.pst', version=2)

## **Step 3. Run PEST++ and perform the prior MC in parallel**
To do this in parallel, we specify the number of workers to be 8 which is equal to the number of cores available on the current machine. Note, the WRF-Hydro model is running on a single core. 

In [ ]:
# run PEST++
pyemu.utils.os_utils.start_workers(worker_dir = "/home/docker/wrf-hydro-training/output/lesson5/Prior_MC_Model_Run", 
                                   exe_rel_path = "pestpp-ies", 
                                   pst_rel_path = "wrfpst.pst", 
                                   num_workers=8, 
                                   worker_root='/home/docker/wrf-hydro-training/output/lesson5/',
                                   master_dir = "/home/docker/wrf-hydro-training/output/lesson5/host",
                                   port=4004, 
                                   verbose = True, 
                                   cleanup = False)

## **Step 4. Let's check the run directory**

In [ ]:
%%bash 
ls  /home/docker/wrf-hydro-training/output/lesson5/

In [ ]:
%%bash 
ls  /home/docker/wrf-hydro-training/output/lesson5/host

The successful finish message of PEST++ could be found in the wrfpst.rmr file.

In [ ]:
%%bash 
cat  /home/docker/wrf-hydro-training/output/lesson5/host/wrfpst.rec | tail -n 5

Let us check the content of the workder folder.

In [ ]:
%%bash 
ls  /home/docker/wrf-hydro-training/output/lesson4/worker_0/

Now we could continue to the next lesson where we construct an objective function, and perform rejection sampling on the prior Monte Carlo model simulation, and submit an iES experiment. 